##**This notebook do the proprocessing for the RGB images that going to enter to the Llava**

PNG Converting

In [1]:
import cv2
import os
from PIL import Image

In [2]:
# Paths
src_dir = r"D:\Y4 Research\datasets\open random images\images"
dst_dir = r"D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted"

os.makedirs(dst_dir, exist_ok=True)

# Process images
for filename in os.listdir(src_dir):
    file_path = os.path.join(src_dir, filename)
    try:
        with Image.open(file_path) as img:
            # Convert to RGB
            img_rgb = img.convert("RGB")
            
            # Save as PNG
            base_name = os.path.splitext(filename)[0]
            save_path = os.path.join(dst_dir, base_name + ".png")
            img_rgb.save(save_path, format="PNG")
            
            print(f"Saved: {save_path}")
    except Exception as e:
        print(f"Skipped {filename}: {e}")

Saved: D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted\000_000_003_3045_1.400.png
Saved: D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted\000_000_004_4394_1.400.png
Saved: D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted\000_000_005_9124_2.png
Saved: D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted\000_000_006_5269_4.400.png
Saved: D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted\000_000_007_0701_1.400.png
Saved: D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted\000_000_020_4061_4.400.png
Saved: D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted\000_000_026_5829_4.400.png
Saved: D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted\000_000_027_7945_1.png
Saved: D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted\000_000_035_8156_7.400.png
Saved: D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\PNG Converted\000_000_036_9527_2.400.pn

Adaptive resizing

In [3]:
from tqdm.notebook import tqdm

In [4]:
# Paths
input_dir = r"D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\subset1_Processing\subset1"
output_dir = r"D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\subset1_Processing\resized"

os.makedirs(output_dir, exist_ok=True)

# Parameters
TARGET_SIZE = 640
PADDING_COLOR = (255, 255, 255)  # neutral white

# Resize + pad function
def resize_with_padding(image, target_size=640, pad_color=(255, 255, 255)):
    w, h = image.size

    # Keep aspect ratio
    if w > h:
        new_w = target_size
        new_h = int(h * target_size / w)
    else:
        new_h = target_size
        new_w = int(w * target_size / h)

    image = image.resize((new_w, new_h), Image.LANCZOS)

    # Create square canvas
    new_img = Image.new("RGB", (target_size, target_size), pad_color)

    paste_x = (target_size - new_w) // 2
    paste_y = (target_size - new_h) // 2
    new_img.paste(image, (paste_x, paste_y))

    return new_img

# Get file list
files = os.listdir(input_dir)

# Process images with progress bar
for file in tqdm(files, desc="Resizing images", unit="image"):
    file_path = os.path.join(input_dir, file)

    try:
        with Image.open(file_path) as img:
            img = img.convert("RGB")
            processed_img = resize_with_padding(img, TARGET_SIZE, PADDING_COLOR)

            save_path = os.path.join(output_dir, os.path.splitext(file)[0] + ".png")
            processed_img.save(save_path, format="PNG")

    except Exception as e:
        print(f"✖ Skipped {file}: {e}")


Resizing images:   0%|          | 0/455 [00:00<?, ?image/s]

Adaptive Image Sharpening

In [5]:
import cv2
import os
import numpy as np
from tqdm import tqdm

# Paths
input_dir = r"D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\subset1_Processing\resized"
output_dir = r"D:\Y4 Research\datasets\Food Images\RGB_Preprocessed\subset1_Processing\sharpened"

os.makedirs(output_dir, exist_ok=True)

def adaptive_sharpen(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Compute blur score
    blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()

    # Decide sharpening strength
    if blur_score < 80:
        alpha = 1.8  # strong sharpening
    elif blur_score < 150:
        alpha = 1.4  # medium sharpening
    else:
        return image  # already sharp

    # Unsharp masking
    blurred = cv2.GaussianBlur(image, (0, 0), sigmaX=1.0)
    sharpened = cv2.addWeighted(
        image, alpha,
        blurred, -(alpha - 1),
        0
    )
    return sharpened

# Process images with progress bar
image_files = [
    f for f in os.listdir(input_dir)
    if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff"))
]

for filename in tqdm(image_files, desc="Sharpening images"):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename)

    img = cv2.imread(input_path)
    if img is None:
        print(f"Failed to read: {filename}")
        continue

    sharpened_img = adaptive_sharpen(img)
    cv2.imwrite(output_path, sharpened_img)

print("Adaptive sharpening completed!")


Sharpening images: 100%|██████████| 455/455 [00:09<00:00, 49.17it/s]

Adaptive sharpening completed!
